In [5]:
import yfinance as yf

# Definindo a carteira de ações
carteira_yf = ['ABEV3.SA', 'B3SA3.SA', 'ELET3.SA', 'GGBR4.SA', 'ITSA4.SA',
               'PETR4.SA', 'RENT3.SA', 'SUZB3.SA', 'VALE3.SA', 'WEGE3.SA']

df = yf.download(carteira_yf, start='2022-08-01', end='2023-08-01')

cotacoes = df.stack(level=1)

cotacoes = cotacoes.reset_index().rename(columns={'Ticker':'Ativo'})

cotacoes = cotacoes[['Date', 'Open', 'High', 'Low', 'Close', 'Ativo']]

del carteira_yf, df

[*********************100%%**********************]  10 of 10 completed


In [1]:
# Importando as bibliotecas
import fundamentus
import pandas as pd

# Definindo a carteira de ações
carteira_fund = ["ABEV3", "B3SA3", "ELET3", "GGBR4", "ITSA4",
                 "PETR4", "RENT3", "SUZB3", "VALE3", "WEGE3"]

# Criando um df com algumas infos da carteira
# Utilizando pd.concat ao invés de append
detalhes_list = [fundamentus.get_detalhes_papel(papel) for papel in carteira_fund]
ind = pd.concat(detalhes_list, axis=0)
ind = ind[['Setor', 'Cotacao', 'Min_52_sem', 'Max_52_sem', 'Valor_de_mercado',
           'Nro_Acoes', 'Patrim_Liq', 'Receita_Liquida_12m', 'Receita_Liquida_3m',
           'Lucro_Liquido_12m', 'Lucro_Liquido_3m']]

# Passando o ticker para uma coluna
ind = ind.reset_index()
ind.rename(columns={'index': 'Ativo'}, inplace=True)

# Alterando colunas object para numeric
colunas = ['Cotacao', 'Min_52_sem', 'Max_52_sem', 'Valor_de_mercado', 'Nro_Acoes', 'Patrim_Liq',
           'Receita_Liquida_12m', 'Receita_Liquida_3m', 'Lucro_Liquido_12m', 'Lucro_Liquido_3m']
ind[colunas] = ind[colunas].apply(pd.to_numeric, errors='coerce', axis=1)

# Criando um novo df com alguns indicadores da carteira
ind_2 = fundamentus.get_resultado_raw().reset_index()
ind_2 = ind_2.query("papel in @carteira_fund")
ind_2 = ind_2[['papel', 'P/L', 'Div.Yield', 'P/VP', 'ROE']].reset_index(drop=True)
ind_2.rename(columns={'papel': 'Ativo', 'Div.Yield': 'DY'}, inplace=True)

# Concatenando os dfs em um só com as infos e indicadores
indicadores = pd.merge(ind, ind_2, on="Ativo")

# Criando uma coluna para LPA (Lucro por Ação) e VPA (Valor Patrimonial por ação)
# para calcular a fórmula de Graham  Valor intrínseco de uma ação (VI = √22,5 x LPA x VPA)
indicadores["LPA"] = (indicadores["Lucro_Liquido_12m"] / indicadores["Nro_Acoes"]).round(2)
indicadores["VPA"] = (indicadores["Patrim_Liq"] / indicadores["Nro_Acoes"]).round(2)

del ind, ind_2, carteira_fund, colunas

2024-06-16 21:13:56,695 [logging.log_init] INFO: LOGLEVEL=INFO
c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\fundamentus\detalhes.py:232: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables_html = pd.read_html(content.text, decimal=",", thousands='.')
c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\fundamentus\detalhes.py:232: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  tables_html = pd.read_html(content.text, decimal=",", thousands='.')
c:\Users\user\AppData\Local\Programs\Python\Python312\Lib\site-packages\fundamentus\detalhes.py:232: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'Str

In [4]:
indicadores.head()

,Ativo,Setor,Cotacao,Min_52_sem,Max_52_sem,Valor_de_mercado,Nro_Acoes,Patrim_Liq,Receita_Liquida_12m,Receita_Liquida_3m,Lucro_Liquido_12m,Lucro_Liquido_3m,P/L,DY,P/VP,ROE,LPA,VPA
0,ABEV3,Bebidas,11.56,11.52,14.89,1.821590e+11,1.575770e+10,8.659430e+10,7.948140e+10,2.027630e+10,1.450260e+10,3.700280e+09,12.56,0.0632,2.10,0.1675,0.92,5.50
1,B3SA3,Serviços Financeiros Diversos,10.75,10.69,14.82,5.962490e+10,5.546500e+09,1.987500e+10,9.927090e+09,2.466320e+09,3.992160e+09,9.495830e+08,14.94,0.0432,3.00,0.2009,0.72,3.58
2,ELET3,Energia Elétrica,35.73,33.41,43.99,8.243270e+10,2.307100e+09,1.125820e+11,3.666730e+10,8.718270e+09,4.389210e+09,3.280510e+08,18.78,0.0113,0.73,0.0390,1.90,48.80
3,GGBR4,Siderurgia e Metalurgia,16.97,16.82,23.07,3.578090e+10,2.108480e+09,5.191940e+10,6.625440e+10,1.621030e+10,6.339430e+09,2.043780e+09,5.64,0.0629,0.69,0.1221,3.01,24.62
4,ITSA4,Intermediários Financeiros,9.70,7.85,10.65,1.001830e+11,1.032810e+10,8.043500e+10,7.607000e+09,1.936000e+09,1.414300e+10,3.475000e+09,7.08,0.0883,1.25,0.1758,1.37,7.79
